In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from numba import jit 

In [2]:
@jit(nopython=True)
def legendre(n,x):
    leg = 1.
    if n == 0:
        leg = 1
    elif n==1:
        leg = x
    else:
        leg_down1 = x; leg_down2 = 1.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            leg_down2 = leg_down1
            leg_down1 = leg

    return leg

@jit(nopython=True)
def dlegendre(n,x):
    dleg = 0.
    if n == 0:
        dleg = 0.
    elif n == 1:
        dleg = 1.
    else:
        leg_down1 = x; leg_down2 = 1.
        dleg_down1 = 1.; dleg_down2 = 0.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            dleg = dleg_down2 + (2*i-1)*leg_down1
            leg_down2 = leg_down1
            leg_down1 = leg
            dleg_down2 = dleg_down1
            dleg_down1 = dleg

    return dleg

@jit(nopython=True)
def gauss_legendre_lobatto(length:int):
    #// define some constants
    tolerance = 4.0 * 1.0e-6
    nnewton_iter = 100
    pi = np.pi
    
    # allocate space
    n = length - 1; #// order of polynomial
    x = np.array([0.0 for i in range(length)])
    w = x * 1.
    if n == 1:
        x[0] = -1.; x[1] = 1.
        w[0] = 1.; w[1] = 1.
    else:
        leg = 0.; dleg = 0.; delta = 0.
        # set end points
        x[0]   = -1.0; x[n] = 1.
        w[0]   =  2./(n*(n+1.)); w[n] =  2./(n*(n+1.))

        for i in range(1,(n+1)//2):
            #// initial guess from an approximate form given by SV Parter (1999)
            x[i] = -np.cos( (i+0.25)*pi/n  - 3/(8*n*pi*(i+0.25)))

            #// newton iteration
            for j in range(nnewton_iter):
                leg = legendre(n+1,x[i]) - legendre(n-1,x[i])
                dleg = dlegendre(n+1,x[i]) - dlegendre(n-1,x[i])
                delta = -leg/dleg
                x[i] += delta
                if (np.abs(delta) <= tolerance * np.abs(x[i]) ): break
            
            x[n-i] = - x[i]
            leg = legendre(n, x[i])
            w[i] = 2./(n*(n+1.)*leg*leg)
            w[n-i] = w[i]

        if n %2 == 0 :
            x[n//2] = 0.
            leg = legendre(n, 0.0)
            w[n//2]  = 2./(n*(n+1.)*leg*leg)
    

    return x,w

@jit(nopython=True)
def lagrange_poly(xi,xctrl):
    nctrl = len(xctrl)
    hprime = np.array([0.0 for i in range(nctrl)])
    h = hprime * 1.0

    #! note: this routine is hit pretty hard by the mesher, optimizing the loops here will be beneficial
    for dgr in range(nctrl):
        prod1 = 1.; prod2 = 1.

        #// lagrangian interpolants
        x0 = xctrl[dgr]
        for i in range(nctrl):
            if i != dgr:
                x = xctrl[i]
                prod1 = prod1*(xi-x)
                prod2 = prod2*(x0-x)

        #//! takes inverse to avoid additional divisions
        #//! (multiplications are cheaper than divisions)
        prod2_inv = 1. / prod2
        h[dgr] = prod1 * prod2_inv

        #// first derivatives
        s = 0.0
        for i in range(nctrl):
            if i != dgr :
                prod3 = 1.0
                for j in range(nctrl):
                    if j != dgr and j != i:
                        prod3 = prod3*(xi-xctrl[j])
                s = s + prod3
        hprime[dgr] = s * prod2_inv
    

    return h,hprime


def init_gll_arrays(NGLLX):

    # gll points and weights
    xgll,wxgll = gauss_legendre_lobatto(NGLLX)

    #// derivative, hprimex[i][j] = l_j'(xi_i)
    hprimex = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        _,hprimex[i,:] = lagrange_poly(xgll[i],xgll)

    #// hprimex_wgllx[i,j] = hprime_x[i,j] * wx[i] 
    hprimex_wgllx = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        for j in range(NGLLX):
            hprimex_wgllx[i,j] = hprimex[i,j] * wxgll[i]


    return xgll,wxgll,hprimex,hprimex_wgllx



In [3]:
@jit(nopython=True)
def get_Q_sls_model(Q):

    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref * 0
    NSLS = len(y)

    for i in range(NSLS):
        y[i] = y_sls_ref[i] / Q 
    dy[0] = 1. + 0.5 * y[0]

    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]

    #// copy to y_sls/w_sls
    w_sls = w_sls_ref * 1. 
    y_sls = dy * y 

    return w_sls,y_sls 

@jit(nopython=True)
def compute_q_sls_model(y_sls,w_sls,om,exact=False):
    Q_ls = 1. 
    nsls = len(y_sls)
    if exact:
        for p in range(nsls):
            Q_ls += y_sls[p] * om**2 / (om**2 + w_sls[p]**2)

    # denom
    Q_demon = 0.
    for p in range(nsls):
        Q_demon += y_sls[p] * om * w_sls[p] / (om**2 + w_sls[p]**2)
    
    return Q_ls / Q_demon
@jit(nopython=True)
def get_sls_modulus_factor(freq,Q):
    om = 2 * np.pi * freq

    w_sls,y_sls = get_Q_sls_model(Q)
    s = np.sum(1j * om * y_sls / (w_sls + 1j * om))

    return s + 1.

@jit(nopython=True)
def get_sls_Q_deriv(freq,Q):
    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref  / Q 

    # corrector
    NSLS = len(y_sls_ref)
    dy[0] = 1. + 0.5 * y[0]
    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]
    dd_dqi = dy * 0
    dd_dqi[0] = 0.5 * y_sls_ref[0]
    for i in range(1,NSLS):
        dd_dqi[i] = dd_dqi[i-1] + (dy[i-1] - 0.5) * y_sls_ref[i-1] + dd_dqi[i-1] * y[i-1] +  0.5 * y_sls_ref[i]
        
    dd_dqi = dd_dqi * y + dy * y_sls_ref
    om = 2 * np.pi * freq
    dsdqi = np.sum(1j * om * dd_dqi /(w_sls_ref + 1j * om))
    s = np.sum(1j * om * y * dy / (w_sls_ref + 1j * om))

    return s + 1., dsdqi

In [35]:
@jit(nopython=True)
def assemble_matrix_diag(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco):
    nglob = np.max(ibool) + 1
    M = np.zeros((nglob),dtype=np.complex128)

    for ispec in range(nspec + 1):
        NGL = NGLL
        if ispec == nspec:
            wg = wgrl * 1.
            NGL = NGRL
        else:
            wg = wgll * 1.

        for i in range(NGL):
            ig = ibool[ispec*NGLL+i]
            sp = get_sls_modulus_factor(1.,Qp[ispec*NGLL+i])
            M[ig] += jaco[ispec] * wg[i] * P[ispec*NGLL+i] * sp 
    
    return M 

@jit(nopython=True)
def assemble_matrix_diag_fd(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef):
    dm = P * 0.0j
    for ispec in range(nspec + 1):
        NGL = NGLL
        if ispec == nspec:
            NGL = NGRL
        
        for i in range(NGL):
            id = ispec * NGLL + i
            p = P[id] * 1. 
            dx = 0.01
            P[id] = p * (1 + dx)
            M = assemble_matrix_diag(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco)
            s1 = np.sum(y * M * x)

            P[id] = p * (1 - dx)
            M = assemble_matrix_diag(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco)
            s2 = np.sum(y * M * x)

            dm[id] = (s1 - s2) / (p * dx * 2)

            # recover
            P[id] = p * 1. 
    
    return dm * coef


@jit(nopython=True)
def assemble_matrix_diag_deriv(Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef):
    dM = Qp * 0.0j

    # element
    xe = np.zeros((NGRL),dtype=np.complex128)
    ye = np.zeros((NGRL),dtype=np.complex128)

    for ispec in range(nspec + 1):
        NGL = NGLL
        if ispec == nspec:
            wg = wgrl * 1.
            NGL = NGRL
        else:
            wg = wgll * 1.

        for i in range(NGL):
            ig = ibool[ispec*NGLL+i]
            xe[i] = x[ig]
            ye[i] = y[ig]

        # compute 
        for k in range(NGL):
            sp = get_sls_modulus_factor(1.,Qp[ispec*NGLL+k])
            dM[ispec*NGLL+k] =  wg[k] * jaco[ispec] * xe[k] * ye[k] * sp * coef
    
    return dM

In [37]:
# read mesh
from scipy.io import FortranFile
fio = FortranFile("mesh.bin","r")
ibool = fio.read_ints('i4')
jaco = fio.read_reals('f8')
xN = fio.read_reals('f8')
xQN = fio.read_reals('f8')
wgll = fio.read_reals('f8')
wgrl = fio.read_reals('f8')
nspec = len(jaco) - 1
NGLL = len(wgll)
NGRL = len(wgrl)

fio.close()

np.random.seed(10)
nglob = np.max(ibool) + 1 
x = np.random.rand(nglob) + np.random.rand(nglob) * 1.0j
y = x * 1.0
coef = 1.04 + 0.04j
dM = assemble_matrix_diag_fd(xN,xQN,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef)
dM1 = assemble_matrix_diag_deriv(xQN,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef)

np.allclose(dM,dM1)

True

In [20]:
import numpy as np 
from scipy.linalg.lapack import dsygv
from scipy.linalg import qz,eigh

In [3]:
np.random.seed(5)
A = np.random.rand(2,2)
A = 0.5 * (A + A.T)
B = np.array([[1.,0.5],[0.5,1.2]])

In [21]:
w,Z = eigh(A,B)
AA,BB,QQ,ZZ = qz(A,B)

In [23]:
display(Z)
display(ZZ/ Z)

array([[-1.10576244,  0.2011152 ],
       [ 0.62408673,  0.81433859]])

array([[0.78757376, 2.44394418],
       [0.78757376, 1.0694194 ]])

In [33]:
A @ ZZ[:,1] - w[1] * B @ ZZ[:,1]

array([-0.15829405,  0.03909349])

In [32]:
np.linalg.norm(ZZ[:,0])

1.0

In [35]:
eigh(B)

(array([0.59009805, 1.60990195]),
 array([[-0.77334214,  0.63398891],
        [ 0.63398891,  0.77334214]]))